In [1]:
!pip install Unidecode
!pip install gensim

     |████████████████████████████████| 241 kB 12.3 MB/s 


In [2]:
import re
import bs4
import string
import itertools
from pathlib import Path
from collections import Counter

import nltk
import gensim 
import numpy as np
import pandas as pd
from nltk import word_tokenize
from unidecode import unidecode
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
def preprocess(sentences):

  new_sentence = ''
  sentences = re.sub(r'<\s*br\s*\/s*>', '', sentences)
  sentences = re.sub(r'\n>', ' ', sentences)
  sentences = re.sub(r'\s+', ' ', sentences)
  sentences = re.sub(r'\.+\s*', '.', sentences)
  sentences = re.sub(r'who\'ll', 'who will', sentences)
  sentences = re.sub(r'[IiyouYousheSHE]\'ll', 'i will', sentences)
  sentences = re.sub(r'[wW]ouldn\'t', 'would not', sentences)
  sentences = re.sub(r'[mM]mustn\'t', 'must not', sentences)
  sentences = re.sub(r'[tT]hat\'s', 'that is', sentences)
  # sentences = re.sub(r'oct', ' ', sentences)
  sentences = sentences.replace('.', ' ')

  for sentence in sentences:
    if sentence.isspace() or sentence.isalpha():
      new_sentence += sentence.lower()

  stopset = stopwords.words('english') + list(string.punctuation)
  corpus = " ".join([word for word in word_tokenize(new_sentence) if word not in stopset])
  cleaned_corpus = unidecode(corpus)

  return cleaned_corpus


def extract_reuters_news(path_file):

  file = open(path_file , 'r').read()
  soup = bs4.BeautifulSoup(file)
  news = [el.text for el in soup.find_all('reuters')] # replace body with content, otherwise bs4 wont find any body other then main body

  return news

file_path = Path('/content/drive/MyDrive/Leapfrog_internship/Project 4/reut2-021.sgm')
if file_path.exists():
  news = extract_reuters_news(file_path)
else:
  Print('-- There is error while reading the file. --')

document = []
for i, doc in enumerate(news):
  document.append(preprocess(doc))


In [4]:
news_dataframe = pd.DataFrame(list(zip(news, document)), columns = ['Initial_corpus','Cleaned_corpus'])
news_dataframe.head()

,Initial_corpus,Cleaned_corpus
0,\n19-OCT-1987 15:37:46.03\n\n\n\n\n\n\n \nF \n...,oct f freute f f bccityfedfinanci cityfed fina...
1,\n19-OCT-1987 15:35:53.55\ncrudeship\nbahraini...,oct crudeship bahrainiranusa freute r f amgulf...
2,\n19-OCT-1987 15:34:40.05\nacq\n\n\n\n\n\n \nF...,oct acq f freute b f bcccrvideosays ccr video ...
3,\n19-OCT-1987 15:32:25.38\n\ncanada\n\n\n\n\n ...,oct canada e f freute b f bcgmgmcanadaunitm gm...
4,\n19-OCT-1987 15:32:11.59\n\ncanada\n\n\n\n\n ...,oct canada e f freute u f bccanadadevelopmentu...


In [5]:
def load_model(file_path):

  model_w2v = gensim.models.KeyedVectors.load_word2vec_format(file_path, binary=True)

  return model_w2v

data_folder = Path("/content/drive/MyDrive/Leapfrog_internship/Project 6/")
model_path = documents_dfh = data_folder / "GoogleNews-vectors-negative300.bin.gz"

model_w2v  = load_model(model_path)



In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances


def get_tokenized_elements():

  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(news_dataframe.Cleaned_corpus)
  tokenized_documents = tokenizer.texts_to_sequences(news_dataframe.Cleaned_corpus)
  tokenized_paded_documents = pad_sequences(tokenized_documents, maxlen=64, padding='post')

  return tokenizer, tokenized_paded_documents

def get_tfidf_elements():

  tfidfvectoriser = TfidfVectorizer(max_features = 3000)
  tfidfvectoriser.fit(news_dataframe.Cleaned_corpus)
  tfidf_vectors = tfidfvectoriser.transform(news_dataframe.Cleaned_corpus)
  tfidf_vectors = tfidf_vectors.toarray()
  words = tfidfvectoriser.get_feature_names()

  return words, tfidf_vectors

def document_word_embeddings():

  tokenizer, tokenized_paded_documents = get_tokenized_elements()
  vocab_size = len(tokenizer.word_index)+1
  embedding_matrix = np.zeros((vocab_size, 300))
  for word, i in tokenizer.word_index.items():
    if word in model_w2v:
      embedding_matrix[i] = model_w2v[word]  
  doc_word_embeddings = np.zeros((len(tokenized_paded_documents), 3000, 300))
  for i in range(len(tokenized_paded_documents)):
      for j in range(len(tokenized_paded_documents[0])):
          doc_word_embeddings[i][j] = embedding_matrix[tokenized_paded_documents[i][j]]

  return embedding_matrix, doc_word_embeddings

def document_embeddings():

  tokenizer, tokenized_paded_documents = get_tokenized_elements()
  words, tfidf_vectors = get_tfidf_elements()
  embedding_matrix, doc_word_embeddings = document_word_embeddings()
  documentEmbeddings = np.zeros((len(tokenized_paded_documents), 300))
  for i in range(len(doc_word_embeddings)):
    for j in range(len(words)):   
      documentEmbeddings[i] += embedding_matrix[tokenizer.word_index[words[j]]] * tfidf_vectors[i][j]       
  documentEmbeddings = documentEmbeddings / np.sum(tfidf_vectors, axis = 1).reshape(-1, 1)
  
  return documentEmbeddings



In [11]:
documentEmbeddings = document_embeddings()

def most_similar(doc_id, similarity_matrix, matrix):

    print (f'Document: {news_dataframe.iloc[doc_id]["Initial_corpus"]}')
    print ('\n')
    print ('Similar Documents:')
    if matrix == 'Cosine Similarity':
        similar_ix = np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix == 'Euclidean Distance':
        similar_ix = np.argsort(similarity_matrix[doc_id])
    for ix in range(5):
        if ix == doc_id:
            continue
        print('\n')
        print (f'Document: {news_dataframe.iloc[ix]["Initial_corpus"]}')
        print (f'{matrix} : {similarity_matrix[doc_id][ix]}')

pairwise_similarities=cosine_similarity(documentEmbeddings)
pairwise_differences=euclidean_distances(documentEmbeddings)

most_similar(0,pairwise_similarities,'Cosine Similarity')
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: 
19-OCT-1987 15:37:46.03






 
F 
f2882reute
f f BC-CITYFED-FINANCI   10-19 0013

******CITYFED FINANCIAL CORP SAYS IT CUT QTRLY DIVIDEND TO ONE CENT FROM 10 CTS/SHR
Blah blah blah.






Similar Documents:


Document: 
19-OCT-1987 15:35:53.55
crudeship
bahrainiranusa




 
Y 
f2873reute
r f AM-GULF-PLATFORM   10-19 0101

HUGE OIL PLATFORMS DOT GULF LIKE BEACONS
    By ASHRAF FOUAD
    BAHRAIN, Oct 19 - Huge oil platforms dot the Gulf like
beacons -- usually lit up like Christmas trees at night.
    One of them, sitting astride the Rostam offshore oilfield,
was all but blown out of the water by U.S. Warships on Monday.
    The Iranian platform, an unsightly mass of steel and
concrete, was a three-tier structure rising 200 feet (60
metres) above the warm waters of the Gulf until four U.S.
Destroyers pumped some 1,000 shells into it.
    The U.S. Defense Department said just 10 pct of one section
of the structure remained.
    U.S. helicopters destroyed three Iranian gunboats